# 🛠️ Tools
In LangChain, Tools are essentially functions or APIs that the language model can call whenever it needs to go beyond plain text generation. You can think of them as the “hands” of the LLM. The model is very good at reasoning and language, but it can’t do things like query a database, run code, or fetch live data on its own. Tools bridge that gap by letting the model interact with the outside world.

Each tool has a name, a short description, and a function behind it. The description is important because it guides the model on when to use the tool.

**For Example**

You might have a tool called “calculator” with the description “use this for arithmetic operations.”


In [ ]:
## google colab
# %pip install langchain openai
# %pip install langchain langchain-community openai
# %pip install langchain_openai

In [12]:
from langchain_core.tools import tool

@tool("calculator", description="Use this tool to do arithmetic calculations and evalute math expressions.")
def calculator(query: str) -> str:
    try:
        return str(eval(query))
    except Exception:
        return "Error in calculation"

### Example 

In [16]:
import os, json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import TypedDict, Optional
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage

In [14]:
load_dotenv()
llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=os.getenv("OPEN_API_KEY"))

In [15]:
# --- Define tools (structured I/O strongly recommended) ---
class CalcInput(BaseModel):
    expression: str = Field(..., description="A valid Python arithmetic expression, e.g. '2 + 2*10'")

@tool("calculator", args_schema=CalcInput)
def calculator(expression: str) -> str:
    """Use this tool to do arithmetic calculations and evalute math expressions."""
    # Do not use eval in real apps; use a proper parser. This is a demo.
    try:
        return str(eval(query))
    except Exception:
        return "Error in calculation"

class WeatherIn(BaseModel):
    city: str = Field(..., description="City name, e.g. 'Dallas'")

@tool("fake_weather", args_schema=WeatherIn)
def fake_weather(city: str) -> str:
    """Return a mock weather string (demo)."""
    return f"Weather in {city}: 95°F, sunny."

tools = [calculator, fake_weather]

# --- Bind tools to the model ---
router = llm.bind_tools(tools)

# --- Simple tool loop: model decides -> we execute -> feed result back ---
def ask(prompt: str):
    # 1) Ask the model
    res = router.invoke([HumanMessage(content=prompt)])

    # 2) If it requested tool(s), run them and return final answer
    msgs = [HumanMessage(content=prompt), res]
    if hasattr(res, "tool_calls") and res.tool_calls:
        for tc in res.tool_calls:
            name = tc["name"]
            args = tc["args"] or {}
            fn = next(t for t in tools if t.name == name)
            output = fn.invoke(args)  # run the tool
            msgs.append(ToolMessage(tool_call_id=tc["id"], content=str(output)))

        # 3) Give tool outputs back to the model for the final answer
        final = llm.invoke(msgs)
        return final.content

    # No tool call—model answered directly
    return res.content

print(ask("What's 24*19? Then tell me the (fake) weather in Dallas."))


The result of \(24 \times 19\) is 456. As for the (fake) weather in Dallas, it's 95°F and sunny.
